In [7]:
"""
Superstore Data Preprocessing Script
------------------------------------
Purpose:
1. Read SuperStore Excel data (Orders, Returns, Users)
2. Clean and normalize data
3. Save cleaned datasets to CSV
4. Import cleaned data into SQLite database with indexes

Author: [POUYA BATHAEI POURMAND]
Date: [2025-8-15]
"""

import os
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine, text

# -------------------------------
# 1. Paths & File Names
# -------------------------------
# Define the directory where the data file is stored
DATA_DIR = Path(r"C:\Users\user\Desktop\superstore_analyse")
# Define the Excel file path
EXCEL_FILE = DATA_DIR / "SuperStoreUS-2015.xlsx"
# Define the SQLite database file path
DB_FILE = DATA_DIR / "sales_dashboard.db"

# -------------------------------
# 2. Read Data from Excel
# -------------------------------
# Load each sheet from the Excel file into separate DataFrames
orders = pd.read_excel(EXCEL_FILE, sheet_name="Orders", engine="openpyxl")
returns = pd.read_excel(EXCEL_FILE, sheet_name="Returns", engine="openpyxl")
users = pd.read_excel(EXCEL_FILE, sheet_name="Users", engine="openpyxl")

# -------------------------------
# 3. Normalize Column Names
# -------------------------------
# Function to clean column names: lowercase, replace spaces/special chars with underscores
def normalize_cols(df):
    cols = (df.columns
              .str.strip()
              .str.lower()
              .str.replace('[^0-9a-zA-Z]+', '_', regex=True)
              .str.replace('_+', '_', regex=True)
              .str.strip('_'))
    df.columns = cols
    return df

orders = normalize_cols(orders)
returns = normalize_cols(returns)
users   = normalize_cols(users)

# -------------------------------
# 4. Clean Data
# -------------------------------
# Remove leading/trailing spaces from string values
orders = orders.applymap(lambda x: x.strip() if isinstance(x, str) else x)
returns = returns.applymap(lambda x: x.strip() if isinstance(x, str) else x)
users   = users.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Fix inconsistent order priority values
if 'order_priority' in orders.columns:
    orders['order_priority'] = orders['order_priority'].replace({'Not Specified': 'Low'})

# Drop rows with missing essential values in Orders
orders = orders.dropna(subset=['sales', 'profit', 'order_date'])

# Ensure numeric columns have correct type
numeric_cols = ['discount', 'unit_price', 'shipping_cost', 'profit', 'quantity_ordered_new', 'sales']
for col in numeric_cols:
    if col in orders.columns:
        orders[col] = pd.to_numeric(orders[col], errors='coerce')

# Add calculated column: total_sales = unit_price * quantity_ordered_new
if all(col in orders.columns for col in ['unit_price', 'quantity_ordered_new']):
    orders['total_sales'] = orders['unit_price'] * orders['quantity_ordered_new']

# -------------------------------
# 5. Save Cleaned CSVs
# -------------------------------
# Save cleaned data into CSV files
orders_file  = r"C:\Users\user\Desktop\superstore_analyse\orders_clean.csv"
returns_file = r"C:\Users\user\Desktop\superstore_analyse\returns_clean.csv"
users_file   = r"C:\Users\user\Desktop\superstore_analyse\users_clean.csv"

orders.to_csv(orders_file, index=False)
returns.to_csv(returns_file, index=False)
users.to_csv(users_file, index=False)

print("Cleaned CSV files saved")

# -------------------------------
# 6. Import to SQLite Database
# -------------------------------
# Create SQLite engine (connection)
engine = create_engine(f"sqlite:///{DB_FILE}", echo=False)

# Load DataFrames into SQLite database tables
orders.to_sql('orders', engine, if_exists='replace', index=False)
returns.to_sql('returns', engine, if_exists='replace', index=False)
users.to_sql('users', engine, if_exists='replace', index=False)

# Create indexes for better query performance
with engine.connect() as conn:
    conn.execute(text('CREATE INDEX IF NOT EXISTS idx_orders_orderid ON orders (order_id);'))
    conn.execute(text('CREATE INDEX IF NOT EXISTS idx_returns_orderid ON returns (order_id);'))
    conn.execute(text('CREATE INDEX IF NOT EXISTS idx_orders_customer ON orders (customer_id);'))
    conn.commit()

print(f"Data successfully imported to SQLite database: {DB_FILE}")


C:\Users\user\AppData\Local\Temp\ipykernel_16808\342940549.py:59: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  orders = orders.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\user\AppData\Local\Temp\ipykernel_16808\342940549.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  returns = returns.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\user\AppData\Local\Temp\ipykernel_16808\342940549.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  users   = users.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Cleaned CSV files saved
Data successfully imported to SQLite database: C:\Users\user\Desktop\superstore_analyse\sales_dashboard.db
